In [6]:
import json
from datasets import load_dataset

dataset_path = 'cerebras/SlimPajama-627B'
subset = 'sample'

# load dataset as iterator with n chunks
dataset = load_dataset(dataset_path, subset,  streaming=True)


Resolving data files:   0%|          | 0/118332 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31428 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31411 [00:00<?, ?it/s]

ValueError: BuilderConfig 'sample' not found. Available: ['default']

In [ ]:
# collect 2500 samples whose ['meta']['language'] is 'en'
samples = []
for sample in dataset['train']:
    sample_meta = json.loads(sample['meta'])
    if sample_meta['language'] == 'en':
        samples.append(sample)
    if len(samples) == 2500:
        break

In [ ]:
# save samples to a file
with open('../data/stability_samples.json', 'w') as f:
    json.dump(samples, f)

In [ ]:
# Example on computing perplexity on sample
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("distilgpt2")
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")

model.eval()
losses = []
for sample in tqdm(samples[:10]):
    content = sample["raw_content"]
    tok_content = tokenizer.encode(content, return_tensors="pt", max_length=128, truncation=True)
    with torch.no_grad():
        outputs = model(tok_content, labels=tok_content)

    losses.append(outputs.loss)

perplexity = torch.exp(
    torch.mean(torch.tensor(losses))
)
print(perplexity.item())


100%|██████████| 10/10 [00:02<00:00,  4.92it/s]


39.35826873779297
